In [127]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.preprocessing.text import Tokenizer

In [128]:
max_words = 5000
batch_size = 128
epochs = 5

Getting data:

In [129]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space']
categories = None
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
from pprint import pprint
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [130]:
newsgroups_train.filenames.shape

(11314,)

In [131]:
newsgroups_train.target.shape

(11314,)

In [132]:
newsgroups_train.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

Converting text to vectors:

In [133]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(newsgroups_train["data"])
x_train = tokenizer.texts_to_matrix(newsgroups_train["data"], mode='binary')
x_test = tokenizer.texts_to_matrix(newsgroups_test["data"], mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (11314, 5000)
x_test shape: (7532, 5000)


In [134]:
num_classes = np.max(newsgroups_train["target"]) + 1
print(num_classes, 'classes')

20 classes


Convert class vector to binary class matrix (for use with categorical_crossentropy):

In [135]:
y_train = keras.utils.to_categorical(newsgroups_train["target"], num_classes)
y_test = keras.utils.to_categorical(newsgroups_test["target"], num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (11314, 20)
y_test shape: (7532, 20)


Building model functionally:

In [136]:
a = Input(shape=(max_words,))
b = Dense(512)(a)
b = Activation('relu')(b)
b = Dropout(0.5)(b)
b = Dense(num_classes)(b)
b = Activation('softmax')(b)
model = Model(inputs=a, outputs=b)

In [137]:
model.layers

In [138]:
print(model.to_yaml())

backend: tensorflow
class_name: Model
config:
  input_layers:
  - [input_9, 0, 0]
  layers:
  - class_name: InputLayer
    config:
      batch_input_shape: !!python/tuple [null, 5000]
      dtype: float32
      name: input_9
      sparse: false
    inbound_nodes: []
    name: input_9
  - class_name: Dense
    config:
      activation: linear
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      kernel_constraint: null
      kernel_initializer:
        class_name: VarianceScaling
        config: {distribution: uniform, mode: fan_avg, scale: 1.0, seed: null}
      kernel_regularizer: null
      name: dense_17
      trainable: true
      units: 512
      use_bias: true
    inbound_nodes:
    - - - input_9
        - 0
        - 0
        - {}
    name: dense_17
  - class_name: Activation
    config: {activation: relu, name: activation_17, trainable: true}
    inbound_nodes:
    - 

In [139]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [140]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 10182 samples, validate on 1132 samples
Epoch 1/5
10182/10182 [==============================] - 11s 1ms/step - loss: 1.5601 - acc: 0.6196 - val_loss: 0.6467 - val_acc: 0.8604
Epoch 2/5
10182/10182 [==============================] - 12s 1ms/step - loss: 0.4438 - acc: 0.9019 - val_loss: 0.4446 - val_acc: 0.8922
Epoch 3/5
10182/10182 [==============================] - 11s 1ms/step - loss: 0.2292 - acc: 0.9539 - val_loss: 0.3786 - val_acc: 0.8940
Epoch 4/5
10182/10182 [==============================] - 11s 1ms/step - loss: 0.1367 - acc: 0.9777 - val_loss: 0.3543 - val_acc: 0.9019
Epoch 5/5
10182/10182 [==============================] - 14s 1ms/step - loss: 0.0868 - acc: 0.9890 - val_loss: 0.3398 - val_acc: 0.9028


In [141]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7532/7532 [==============================] - 2s 306us/step


Test score: 0.7093810075087809
Test accuracy: 0.8043016463090813
